In [7]:
include("main_whit.jl")

LU_decompose (generic function with 1 method)

In [9]:
n = 8
D = ddmat(1:n, 2)
display(D)

M = diagm(ones(n)) + λ * D' * D
# ddmat(ddmat(1:n, 1), 1)

6×8 SparseMatrixCSC{Int64, Int64} with 18 stored entries:
 1  -2   1   ⋅   ⋅   ⋅   ⋅  ⋅
 ⋅   1  -2   1   ⋅   ⋅   ⋅  ⋅
 ⋅   ⋅   1  -2   1   ⋅   ⋅  ⋅
 ⋅   ⋅   ⋅   1  -2   1   ⋅  ⋅
 ⋅   ⋅   ⋅   ⋅   1  -2   1  ⋅
 ⋅   ⋅   ⋅   ⋅   ⋅   1  -2  1

8×8 Matrix{Num}:
   1.0 + λ  -2λ           λ         …    0.0         0.0         0.0
 -2λ          1.0 + 5λ  -4λ              0.0         0.0         0.0
   λ        -4λ           1.0 + 6λ       0.0         0.0         0.0
   0.0        λ         -4λ              λ           0.0         0.0
   0.0        0.0         λ            -4λ           λ           0.0
   0.0        0.0         0.0       …    1.0 + 6λ  -4λ           λ
   0.0        0.0         0.0          -4λ           1.0 + 5λ  -2λ
   0.0        0.0         0.0            λ         -2λ           1.0 + λ

In [10]:
U = deepcopy(M)
L = typeof(U)(diagm(ones(n)))

# 仅运行三次，猜测公式的形式
for i = 1:3
    r1 = U[i, :]
    for j = i+1:min(i+2, n)
        f = U[j, i] / U[i, i]
        L[j, i] = f
        U[j, :] .= U[j, :] .- (f * r1)
    end
end
## U
## Elier采用的是LU分解 
display(L)
display(U)


8×8 Matrix{Num}:
          1.0       …  0.0  0.0  0.0  0.0  0.0
 (-2λ) / (1.0 + λ)     0.0  0.0  0.0  0.0  0.0
     λ / (1.0 + λ)     0.0  0.0  0.0  0.0  0.0
          0.0          1.0  0.0  0.0  0.0  0.0
          0.0          0.0  1.0  0.0  0.0  0.0
          0.0       …  0.0  0.0  1.0  0.0  0.0
          0.0          0.0  0.0  0.0  1.0  0.0
          0.0          0.0  0.0  0.0  0.0  1.0

8×8 Matrix{Num}:
 1.0 + λ  …    0.0         0.0         0.0
 0             0.0         0.0         0.0
 0             0.0         0.0         0.0
 0.0           λ           0.0         0.0
 0.0         -4λ           λ           0.0
 0.0      …    1.0 + 6λ  -4λ           λ
 0.0         -4λ           1.0 + 5λ  -2λ
 0.0           λ         -2λ           1.0 + λ

In [ ]:
# U = deepcopy(M)
# L,U = LU_decompose(U)
# L

In [11]:
r = lu(M)
## lu采用for循环的形式，
r.L
# r.U

8×8 Matrix{Num}:
          1         …  0
 (-2λ) / (1.0 + λ)     0
     λ / (1.0 + λ)     0
          0.0          0
          0.0          0
          0.0       …  0
          0.0          0
          0.0          1